<a href="https://colab.research.google.com/github/Natsumisnooker/Machine-Learning/blob/main/ML_Reproduced_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libary

In [5]:
# ติดตั้ง required packages
print("📦 ติดตั้ง packages ที่จำเป็น...")
!pip install xlsxwriter
!pip install textstat

import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from textstat import flesch_reading_ease, smog_index, dale_chall_readability_score, coleman_liau_index, gunning_fog
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import io
import os
from datetime import datetime
import xlsxwriter

print("✅ ติดตั้ง packages เสร็จสิ้น")

# ดาวน์โหลดข้อมูล NLTK ที่จำเป็น
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

class StylometricFeatureExtractor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.imperative_verbs = ['click', 'verify', 'submit', 'download', 'update']
        self.modal_verbs = ['can', 'could', 'should', 'must', 'will', 'would']
        self.uncertainty_adverbs = ['maybe', 'possibly', 'perhaps']
        self.technical_jargon = ['security', 'account', 'update', 'technical', 'password', 'login']
        self.promotional_words = ['offer', 'deal', 'free', 'discount', 'limited', 'exclusive']
        self.politeness_markers = ['please', 'thank', 'appreciate', 'kindly']
        self.aggressiveness_markers = ['must', 'now', 'immediately']
        self.urgency_markers = ['urgent', 'asap', 'immediately', 'urgently']
        self.conditional_phrases = ['if', 'unless']
        self.personalization_markers = ['you', 'your']
        self.conjunctions = ['and', 'but', 'or', 'because']
        self.prepositions = ['in', 'on', 'at', 'by', 'with']
        self.function_words = ['the', 'is', 'at', 'which', 'on']
        self.pronouns = ['i', 'you', 'he', 'she', 'it', 'we', 'they']

    def count_syllables(self, word):
        """นับจำนวน syllables ในคำ"""
        word = word.lower()
        count = 0
        vowels = "aeiouy"
        if word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith("e"):
            count -= 1
        if count == 0:
            count += 1
        return count

    def extract_all_features(self, text):
        """Extract all 60 stylometric features จากข้อความ"""
        features = {}

        # Basic text cleaning
        text = str(text)
        words = word_tokenize(text)
        sentences = sent_tokenize(text)

        # Filter out non-alphanumeric words for some features
        words_clean = [word for word in words if word.isalnum()]

        # 1. Lexical Features (16.7%)
        features['word_count'] = len(words_clean)
        features['character_count'] = len(text)
        features['average_word_length'] = np.mean([len(word) for word in words_clean]) if words_clean else 0
        features['sentence_count'] = len(sentences)
        features['average_sentence_length'] = len(words_clean) / len(sentences) if sentences else 0
        features['unique_word_count'] = len(set(words_clean))
        features['lexical_diversity'] = features['unique_word_count'] / features['word_count'] if features['word_count'] > 0 else 0
        features['number_of_emails'] = text.count('@')
        features['uppercase_word_count'] = sum(1 for word in words_clean if word.isupper())
        features['uppercase_word_ratio'] = features['uppercase_word_count'] / features['word_count'] if features['word_count'] > 0 else 0
        features['complex_words_count'] = sum(1 for word in words_clean if len(word) > 6)
        features['average_syllables_per_word'] = np.mean([self.count_syllables(word) for word in words_clean]) if words_clean else 0

        # 2. Syntactic Features (18.3%)
        features['comma_count'] = text.count(',')
        features['semicolon_count'] = text.count(';')
        features['colon_count'] = text.count(':')
        features['exclamation_count'] = text.count('!')
        features['quotation_count'] = text.count('"')
        features['dash_count'] = text.count('-')

        # Sentence complexity
        conjunction_count = sum(text.lower().count(conj) for conj in self.conjunctions)
        features['sentence_complexity_ratio'] = conjunction_count / features['sentence_count'] if features['sentence_count'] > 0 else 0
        features['clause_density'] = features['sentence_complexity_ratio'] / features['sentence_count'] if features['sentence_count'] > 0 else 0

        # Density features
        pronoun_count = sum(text.lower().count(pronoun) for pronoun in self.pronouns)
        features['pronoun_density'] = pronoun_count / features['word_count'] if features['word_count'] > 0 else 0

        preposition_count = sum(text.lower().count(prep) for prep in self.prepositions)
        features['preposition_density'] = preposition_count / features['word_count'] if features['word_count'] > 0 else 0

        function_word_count = sum(text.lower().count(func) for func in self.function_words)
        features['function_word_density'] = function_word_count / features['word_count'] if features['word_count'] > 0 else 0

        # 3. Punctuation Features (16.7%)
        punctuation_chars = [',', ';', ':', '!', '"', '-', '.', '?', "'", '(', ')', '[', ']', '{', '}']
        for char in punctuation_chars:
            features[f'punctuation_{char}_freq'] = text.count(char) / len(text) if len(text) > 0 else 0

        features['punctuation_variety'] = len(set(char for char in text if char in punctuation_chars))

        # 4. Readability Scores (16.7%)
        try:
            features['flesch_reading_ease'] = flesch_reading_ease(text)
            features['smog_index'] = smog_index(text)
            features['dale_chall_score'] = dale_chall_readability_score(text)
            features['coleman_liau_index'] = coleman_liau_index(text)
            features['gunning_fog_index'] = gunning_fog(text)
        except:
            features['flesch_reading_ease'] = 0
            features['smog_index'] = 0
            features['dale_chall_score'] = 0
            features['coleman_liau_index'] = 0
            features['gunning_fog_index'] = 0

        # 5. Word Category Features (11.7%)
        features['number_of_pronouns'] = pronoun_count
        features['first_person_pronoun_count'] = sum(text.lower().count(pronoun) for pronoun in ['i', 'we', 'my', 'our'])
        features['second_person_pronoun_count'] = sum(text.lower().count(pronoun) for pronoun in ['you', 'your'])
        features['imperative_verbs_count'] = sum(text.lower().count(verb) for verb in self.imperative_verbs)
        features['modal_verbs_count'] = sum(text.lower().count(verb) for verb in self.modal_verbs)
        features['uncertainty_adverbs_count'] = sum(text.lower().count(adverb) for adverb in self.uncertainty_adverbs)
        features['technical_jargon_count'] = sum(text.lower().count(jargon) for jargon in self.technical_jargon)
        features['promotional_words_count'] = sum(text.lower().count(word) for word in self.promotional_words)

        # 6. Email-Specific Features (6.7%)
        features['attachment_mentions'] = text.lower().count('attachment')

        # 7. Complexity Features (5.0%)
        # Bigram and trigram counts
        vectorizer_bigram = CountVectorizer(ngram_range=(2, 2), max_features=100)
        vectorizer_trigram = CountVectorizer(ngram_range=(3, 3), max_features=100)

        try:
            bigram_matrix = vectorizer_bigram.fit_transform([text])
            trigram_matrix = vectorizer_trigram.fit_transform([text])
            features['bigram_count'] = bigram_matrix.sum()
            features['trigram_count'] = trigram_matrix.sum()
        except:
            features['bigram_count'] = 0
            features['trigram_count'] = 0

        word_lengths = [len(word) for word in words_clean]
        features['word_length_variation'] = np.std(word_lengths) if word_lengths else 0

        # 8. Stylistic Features (8.3%)
        features['politeness_markers_count'] = sum(text.lower().count(marker) for marker in self.politeness_markers)
        features['aggressiveness_markers_count'] = sum(text.lower().count(marker) for marker in self.aggressiveness_markers)
        features['urgency_markers_count'] = sum(text.lower().count(marker) for marker in self.urgency_markers)
        features['conditional_phrases_count'] = sum(text.lower().count(phrase) for phrase in self.conditional_phrases)
        features['personalization_markers_count'] = sum(text.lower().count(marker) for marker in self.personalization_markers)

        return features

def create_complete_excel_report(train_df, test_df, train_features_df, test_features_df,
                                results_df, feature_importance_df, ablation_df, model_name):
    """สร้างไฟล์ Excel รายงานละเอียดแบบครบถ้วน รวม Subject และ Body"""

    # สร้าง Excel writer
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'complete_stylometric_analysis_report_{timestamp}.xlsx'

    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:

        # 1. สรุปข้อมูลพื้นฐาน
        summary_data = {
            'รายการ': [
                'จำนวนข้อมูลฝึก',
                'จำนวนข้อมูลทดสอบ',
                'จำนวน Features',
                'วันที่วิเคราะห์',
                'โมเดลที่ดีที่สุด',
                'ความแม่นยำสูงสุด'
            ],
            'ค่า': [
                len(train_df),
                len(test_df),
                train_features_df.shape[1],
                datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                model_name,
                f"{results_df.loc[model_name, 'Accuracy']:.4f}"
            ]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='สรุปผล', index=False)

        # 2. ข้อมูลต้นฉบับ + Features (Train)
        train_complete = train_df.copy()
        # เพิ่มคอลัมน์ combined_text ถ้ายังไม่มี
        if 'combined_text' not in train_complete.columns:
            if 'Subject' in train_complete.columns and 'Body' in train_complete.columns:
                train_complete['combined_text'] = train_complete['Subject'] + " " + train_complete['Body']

        # รวม features กับข้อมูลต้นฉบับ
        for feature in train_features_df.columns:
            train_complete[feature] = train_features_df[feature].values

        train_complete.to_excel(writer, sheet_name='ข้อมูลต้นฉบับ_Train', index=False)

        # 3. ข้อมูลต้นฉบับ + Features (Test)
        test_complete = test_df.copy()
        # เพิ่มคอลัมน์ combined_text ถ้ายังไม่มี
        if 'combined_text' not in test_complete.columns:
            if 'Subject' in test_complete.columns and 'Body' in test_complete.columns:
                test_complete['combined_text'] = test_complete['Subject'] + " " + test_complete['Body']

        # รวม features กับข้อมูลต้นฉบับ
        for feature in test_features_df.columns:
            test_complete[feature] = test_features_df[feature].values

        test_complete.to_excel(writer, sheet_name='ข้อมูลต้นฉบับ_Test', index=False)

        # 4. สถิติ Features ของ Train Data
        train_stats = train_features_df.describe().T
        train_stats['feature'] = train_stats.index
        train_stats = train_stats[['feature', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
        train_stats.to_excel(writer, sheet_name='สถิติFeatures_Train', index=False)

        # 5. สถิติ Features ของ Test Data
        test_stats = test_features_df.describe().T
        test_stats['feature'] = test_stats.index
        test_stats = test_stats[['feature', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
        test_stats.to_excel(writer, sheet_name='สถิติFeatures_Test', index=False)

        # 6. ผลลัพธ์โมเดล
        results_df.to_excel(writer, sheet_name='ผลลัพธ์โมเดล', index=True)

        # 7. Feature Importance
        if feature_importance_df is not None:
            feature_importance_df.to_excel(writer, sheet_name='Feature_Importance', index=False)

        # 8. Ablation Study
        if ablation_df is not None:
            ablation_df.to_excel(writer, sheet_name='Ablation_Study', index=True)

        # 9. Comparison between Train and Test Features
        comparison_data = []
        for feature in train_features_df.columns:
            train_mean = train_features_df[feature].mean()
            test_mean = test_features_df[feature].mean()
            train_std = train_features_df[feature].std()
            test_std = test_features_df[feature].std()

            comparison_data.append({
                'Feature': feature,
                'Train_Mean': train_mean,
                'Test_Mean': test_mean,
                'Train_STD': train_std,
                'Test_STD': test_std,
                'Mean_Difference': abs(train_mean - test_mean),
                'STD_Difference': abs(train_std - test_std)
            })

        comparison_df = pd.DataFrame(comparison_data)
        comparison_df.to_excel(writer, sheet_name='เปรียบเทียบFeatures', index=False)

        # 10. ตัวอย่างอีเมลและ Features (5 อันดับแรก)
        sample_train = train_complete.head().copy()
        sample_train.to_excel(writer, sheet_name='ตัวอย่างอีเมล_Train', index=False)

        sample_test = test_complete.head().copy()
        sample_test.to_excel(writer, sheet_name='ตัวอย่างอีเมล_Test', index=False)

    return filename

def create_model_performance_excel(results_df, confusion_matrices, feature_importance_df, ablation_df, model_name):
    """สร้างไฟล์ Excel สรุปผลการทดลอง"""

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'model_performance_summary_{timestamp}.xlsx'

    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:

        # 1. ผลลัพธ์หลัก
        results_df.to_excel(writer, sheet_name='Model_Results', index=True)

        # 2. Confusion Matrices
        cm_data = []
        for model_name, cm in confusion_matrices.items():
            tn, fp, fn, tp = cm.ravel()
            cm_data.append({
                'Model': model_name,
                'True_Negative': tn,
                'False_Positive': fp,
                'False_Negative': fn,
                'True_Positive': tp,
                'Accuracy': results_df.loc[model_name, 'Accuracy']
            })
        cm_df = pd.DataFrame(cm_data)
        cm_df.to_excel(writer, sheet_name='Confusion_Matrices', index=False)

        # 3. Feature Importance
        if feature_importance_df is not None:
            feature_importance_df.to_excel(writer, sheet_name='Feature_Importance', index=False)

        # 4. Ablation Study
        if ablation_df is not None:
            ablation_df.to_excel(writer, sheet_name='Ablation_Study', index=True)

        # 5. Model Comparison
        comparison_data = {
            'Metric': ['Best Accuracy', 'Best Precision', 'Best Recall', 'Best F1-Score', 'Best AUC'],
            'Value': [
                results_df['Accuracy'].max(),
                results_df['Precision'].max(),
                results_df['Recall'].max(),
                results_df['F1-Score'].max(),
                results_df['AUC-Score'].max()
            ],
            'Best_Model': [
                results_df['Accuracy'].idxmax(),
                results_df['Precision'].idxmax(),
                results_df['Recall'].idxmax(),
                results_df['F1-Score'].idxmax(),
                results_df['AUC-Score'].idxmax()
            ]
        }
        comparison_df = pd.DataFrame(comparison_data)
        comparison_df.to_excel(writer, sheet_name='Model_Comparison', index=False)

    return filename

def create_feature_statistics_csv(train_features_df, test_features_df, filename_prefix):
    """สร้างไฟล์ CSV สถิติ features"""

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # สรุปสถิติ features
    train_stats = train_features_df.describe().T
    train_stats['dataset'] = 'Train'
    train_stats['feature'] = train_stats.index

    test_stats = test_features_df.describe().T
    test_stats['dataset'] = 'Test'
    test_stats['feature'] = test_stats.index

    # รวมสถิติ
    combined_stats = pd.concat([train_stats, test_stats])
    combined_stats = combined_stats[['feature', 'dataset', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]

    # บันทึกไฟล์
    stats_filename = f'{filename_prefix}_feature_statistics_{timestamp}.csv'
    combined_stats.to_csv(stats_filename, index=False)

    return stats_filename

# อัพโหลดไฟล์
print("📁 กรุณาอัพโหลดไฟล์ Train.csv และ Test.csv")
uploaded = files.upload()

# แสดงชื่อไฟล์ที่อัพโหลดได้
print("\n📂 ไฟล์ที่อัพโหลด:")
for filename in uploaded.keys():
    print(f"  - {filename}")

# หาชื่อไฟล์ที่ถูกต้องโดยอัตโนมัติ
train_filename = None
test_filename = None

for filename in uploaded.keys():
    if 'train' in filename.lower():
        train_filename = filename
    elif 'test' in filename.lower():
        test_filename = filename

# ถ้าหาไม่เจอให้ใช้ชื่อแรกที่พบ
if not train_filename:
    train_filename = list(uploaded.keys())[0]
if not test_filename and len(uploaded.keys()) > 1:
    test_filename = list(uploaded.keys())[1]

print(f"\n🔍 ใช้ไฟล์:")
print(f"  Train: {train_filename}")
print(f"  Test: {test_filename}")

# โหลดข้อมูลจากไฟล์ที่อัพโหลด
train_df = pd.read_csv(io.BytesIO(uploaded[train_filename]))
test_df = pd.read_csv(io.BytesIO(uploaded[test_filename]))

# แสดงข้อมูลพื้นฐาน
print("\n" + "=" * 80)
print("📊 ข้อมูลพื้นฐานของ Dataset")
print("=" * 80)
print(f"Train dataset shape: {train_df.shape}")
print(f"Test dataset shape: {test_df.shape}")

print("\nคอลัมน์ใน Train dataset:")
print(train_df.columns.tolist())

# ตรวจสอบโครงสร้างข้อมูล
print("\n" + "=" * 80)
print("🔍 การตรวจสอบโครงสร้างข้อมูล")
print("=" * 80)

# ตรวจสอบว่ามีคอลัมน์ 'Subject' และ 'Body' หรือไม่
has_subject = 'Subject' in train_df.columns
has_body = 'Body' in train_df.columns
has_label = 'Label' in train_df.columns

print(f"มีคอลัมน์ Subject: {has_subject}")
print(f"มีคอลัมน์ Body: {has_body}")
print(f"มีคอลัมน์ Label: {has_label}")

# สร้างคอลัมน์ combined_text สำหรับการ extract features
if has_subject and has_body:
    train_df['combined_text'] = train_df['Subject'].astype(str) + " " + train_df['Body'].astype(str)
    test_df['combined_text'] = test_df['Subject'].astype(str) + " " + test_df['Body'].astype(str)
    print("✅ สร้าง combined_text จาก Subject และ Body")
else:
    print("⚠️  ไม่พบคอลัมน์ 'Subject' หรือ 'Body' จะใช้คอลัมน์แรกเป็นข้อความแทน")
    text_column = train_df.columns[0]
    print(f"📝 ใช้คอลัมน์ '{text_column}' เป็นข้อความ")
    train_df['combined_text'] = train_df[text_column].astype(str)
    test_df['combined_text'] = test_df[text_column].astype(str)

# Extract Stylometric Features
print("\n" + "=" * 80)
print("🔧 การ Extract Stylometric Features")
print("=" * 80)

feature_extractor = StylometricFeatureExtractor()

print("กำลัง extract features จาก train dataset...")
train_features = []
for idx, text in enumerate(train_df['combined_text']):
    if idx % 10 == 0:
        print(f"Processed {idx}/{len(train_df)} emails...")
    features = feature_extractor.extract_all_features(text)
    train_features.append(features)

print("กำลัง extract features จาก test dataset...")
test_features = []
for idx, text in enumerate(test_df['combined_text']):
    if idx % 10 == 0:
        print(f"Processed {idx}/{len(test_df)} emails...")
    features = feature_extractor.extract_all_features(text)
    test_features.append(features)

# แปลงเป็น DataFrame
X_train = pd.DataFrame(train_features)
X_test = pd.DataFrame(test_features)

y_train = train_df['Label']
y_test = test_df['Label']

print(f"\n✅ การ extract features เสร็จสมบูรณ์")
print(f"จำนวน features ที่ได้: {X_train.shape[1]}")
print(f"จำนวนข้อมูลฝึก: {X_train.shape[0]}")
print(f"จำนวนข้อมูลทดสอบ: {X_test.shape[0]}")

# บันทึกไฟล์ CSV ของ Features
print("\n💾 กำลังบันทึกไฟล์ Features...")

# 1. Features ดิบ
X_train.to_csv('stylometric_features_train.csv', index=False)
X_test.to_csv('stylometric_features_test.csv', index=False)

# 2. Features พร้อม Labels
X_train_with_label = X_train.copy()
X_train_with_label['Label'] = y_train.values
X_test_with_label = X_test.copy()
X_test_with_label['Label'] = y_test.values

X_train_with_label.to_csv('stylometric_features_train_with_labels.csv', index=False)
X_test_with_label.to_csv('stylometric_features_test_with_labels.csv', index=False)

# 3. สร้างไฟล์สถิติ features
print("กำลังสร้างไฟล์สถิติ features...")
stats_file = create_feature_statistics_csv(X_train, X_test, 'stylometric')
print(f"✅ สร้างไฟล์สถิติ: {stats_file}")

print("✅ บันทึกไฟล์ Features เรียบร้อยแล้ว")

# เติมค่าที่หายไปด้วยค่าเฉลี่ย
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Encode target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

class_names = label_encoder.classes_
print(f"คลาส: {class_names}")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# กำหนดโมเดล
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': SVC(random_state=42, probability=True),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# ฝึกและประเมินโมเดล
print("\n" + "=" * 80)
print("🤖 การฝึกและประเมินโมเดล")
print("=" * 80)

results = {}
confusion_matrices = {}
trained_models = {}

for name, model in models.items():
    print(f"\n🎯 กำลังฝึกโมเดล: {name}")

    # ฝึกโมเดล
    model.fit(X_train_scaled, y_train_encoded)
    trained_models[name] = model

    # ทำนาย
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

    # คำนวณเมตริก
    accuracy = accuracy_score(y_test_encoded, y_pred)
    precision = precision_score(y_test_encoded, y_pred, average='weighted')
    recall = recall_score(y_test_encoded, y_pred, average='weighted')
    f1 = f1_score(y_test_encoded, y_pred, average='weighted')
    auc = roc_auc_score(y_test_encoded, y_pred_proba)

    # บันทึกผลลัพธ์
    results[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'AUC-Score': auc
    }

    # บันทึก confusion matrix
    cm = confusion_matrix(y_test_encoded, y_pred)
    confusion_matrices[name] = cm

    print(f"✅ {name} Results:")
    print(f"   Accuracy:  {accuracy:.4f}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1-Score:  {f1:.4f}")
    print(f"   AUC-Score: {auc:.4f}")

# แสดงผลลัพธ์ในรูปแบบตาราง
print("\n" + "=" * 80)
print("📈 สรุปผลลัพธ์ทั้งหมด")
print("=" * 80)
results_df = pd.DataFrame(results).T
print(results_df.round(4))

# บันทึกผลลัพธ์โมเดลเป็น CSV
results_df.to_csv('model_performance_results.csv', index=True)
print("✅ บันทึกผลลัพธ์โมเดลเป็น CSV เรียบร้อยแล้ว")

# หาโมเดลที่ดีที่สุด
best_model_name = results_df['Accuracy'].idxmax()
best_accuracy = results_df['Accuracy'].max()

# Feature Importance Analysis
print("\n" + "=" * 80)
print("🔍 การวิเคราะห์ Feature Importance")
print("=" * 80)

xgb_feature_importance_df = None
rf_feature_importance_df = None

# XGBoost Feature Importance
if 'XGBoost' in trained_models:
    xgb_model = trained_models['XGBoost']
    xgb_importance = xgb_model.feature_importances_

    xgb_feature_importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': xgb_importance
    }).sort_values('Importance', ascending=False)

    print("\n📊 Top 15 Feature Importances (XGBoost):")
    print(xgb_feature_importance_df.head(15))

    # บันทึก feature importance
    xgb_feature_importance_df.to_csv('xgb_feature_importance.csv', index=False)
    print("✅ บันทึก XGBoost feature importance เรียบร้อยแล้ว")

# Ablation Study สำหรับ XGBoost
print("\n" + "=" * 80)
print("🔬 Ablation Study - XGBoost")
print("=" * 80)

ablation_df = None

if 'XGBoost' in trained_models and xgb_feature_importance_df is not None:
    # เรียงลำดับ feature ตาม importance
    sorted_features = xgb_feature_importance_df.sort_values('Importance', ascending=False)

    # ทดลองลบ features ตามลำดับ
    feature_sets = {
        'All Features': list(range(X_train.shape[1])),
        'Without Top 10': sorted_features.index[10:].tolist(),
        'Without Top 20': sorted_features.index[20:].tolist(),
        'Without Top 30': sorted_features.index[30:].tolist() if X_train.shape[1] > 30 else sorted_features.index[20:].tolist()
    }

    ablation_results = {}

    for set_name, feature_indices in feature_sets.items():
        if not feature_indices:
            continue

        X_train_subset = X_train_scaled[:, feature_indices]
        X_test_subset = X_test_scaled[:, feature_indices]

        xgb_ablated = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
        xgb_ablated.fit(X_train_subset, y_train_encoded)

        y_pred_ablated = xgb_ablated.predict(X_test_subset)

        accuracy_ablated = accuracy_score(y_test_encoded, y_pred_ablated)
        precision_ablated = precision_score(y_test_encoded, y_pred_ablated, average='weighted')
        recall_ablated = recall_score(y_test_encoded, y_pred_ablated, average='weighted')
        f1_ablated = f1_score(y_test_encoded, y_pred_ablated, average='weighted')

        ablation_results[set_name] = {
            'Accuracy': accuracy_ablated,
            'Precision': precision_ablated,
            'Recall': recall_ablated,
            'F1-Score': f1_ablated,
            'Num Features': len(feature_indices)
        }

    ablation_df = pd.DataFrame(ablation_results).T
    print("\nAblation Study Results:")
    print(ablation_df.round(4))

    # บันทึก ablation study
    ablation_df.to_csv('ablation_study_results.csv', index=True)
    print("✅ บันทึก ablation study results เรียบร้อยแล้ว")

# สร้างไฟล์รายงาน Excel แบบครบถ้วน
print("\n" + "=" * 80)
print("📊 การสร้างไฟล์รายงาน Excel แบบครบถ้วน")
print("=" * 80)

# 1. สร้างรายงานละเอียดแบบครบถ้วน (รวม Subject และ Body)
print("กำลังสร้างรายงานแบบครบถ้วน...")
complete_report_file = create_complete_excel_report(
    train_df, test_df, X_train, X_test,
    results_df, xgb_feature_importance_df, ablation_df, best_model_name
)

# 2. สร้างรายงานสรุปผลการทดลอง
print("กำลังสร้างรายงานสรุปผลการทดลอง...")
performance_report_file = create_model_performance_excel(
    results_df, confusion_matrices, xgb_feature_importance_df, ablation_df, best_model_name
)

print("✅ สร้างไฟล์รายงานเรียบร้อยแล้ว:")
print(f"  - {complete_report_file} (รายงานแบบครบถ้วน)")
print(f"  - {performance_report_file} (รายงานสรุปผล)")

# ดาวน์โหลดไฟล์ทั้งหมด
print("\n📥 กำลังดาวน์โหลดไฟล์...")
files_to_download = [
    'stylometric_features_train.csv',
    'stylometric_features_test.csv',
    'stylometric_features_train_with_labels.csv',
    'stylometric_features_test_with_labels.csv',
    stats_file,
    'model_performance_results.csv',
    complete_report_file,
    performance_report_file
]

# เพิ่มไฟล์ feature importance และ ablation study ถ้ามี
if xgb_feature_importance_df is not None:
    files_to_download.append('xgb_feature_importance.csv')
if ablation_df is not None:
    files_to_download.append('ablation_study_results.csv')

for file in files_to_download:
    if os.path.exists(file):
        files.download(file)
        print(f"  ✅ ดาวน์โหลด {file}")
    else:
        print(f"  ⚠️  ไม่พบไฟล์ {file}")

# สรุปผลลัพธ์
print("\n" + "=" * 80)
print("🎯 สรุปผลการทดลอง")
print("=" * 80)
print(f"🏆 โมเดลที่ดีที่สุด: {best_model_name}")
print(f"📊 ความแม่นยำสูงสุด: {best_accuracy:.4f}")
print(f"🔢 จำนวน features ที่ใช้: {X_train.shape[1]}")
print(f"📁 จำนวนข้อมูลฝึก: {X_train.shape[0]}")
print(f"📁 จำนวนข้อมูลทดสอบ: {X_test.shape[0]}")
print(f"🏷️  คลาส: {class_names.tolist()}")

print(f"\n💾 ไฟล์ที่สร้าง:")
print("  ✅ stylometric_features_train.csv - Features จากข้อมูลฝึก")
print("  ✅ stylometric_features_test.csv - Features จากข้อมูลทดสอบ")
print("  ✅ stylometric_features_train_with_labels.csv - Features + Labels จากข้อมูลฝึก")
print("  ✅ stylometric_features_test_with_labels.csv - Features + Labels จากข้อมูลทดสอบ")
print(f"  ✅ {stats_file} - สถิติ features")
print("  ✅ model_performance_results.csv - ผลลัพธ์โมเดล")
if xgb_feature_importance_df is not None:
    print("  ✅ xgb_feature_importance.csv - Feature importance (XGBoost)")
if ablation_df is not None:
    print("  ✅ ablation_study_results.csv - ผล ablation study")
print(f"  ✅ {complete_report_file} - รายงานแบบครบถ้วน (รวม Subject และ Body)")
print(f"  ✅ {performance_report_file} - รายงานสรุปผลการทดลอง")

print("\n📋 ในไฟล์รายงานแบบครบถ้วนจะมี:")
print("  - ข้อมูลต้นฉบับ (Subject, Body) + Features ทั้งหมด")
print("  - สถิติ features")
print("  - ผลลัพธ์โมเดล")
print("  - Feature importance")
print("  - Ablation study")
print("  - ตัวอย่างอีเมล")

print("\n🎉 การทดลองเสร็จสมบูรณ์! สามารถดาวน์โหลดไฟล์ทั้งหมดได้แล้ว")

📦 ติดตั้ง packages ที่จำเป็น...
✅ ติดตั้ง packages เสร็จสิ้น


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


📁 กรุณาอัพโหลดไฟล์ Train.csv และ Test.csv


Saving Train.csv to Train (1).csv
Saving Test.csv to Test (1).csv

📂 ไฟล์ที่อัพโหลด:
  - Train (1).csv
  - Test (1).csv

🔍 ใช้ไฟล์:
  Train: Train (1).csv
  Test: Test (1).csv

📊 ข้อมูลพื้นฐานของ Dataset
Train dataset shape: (100, 3)
Test dataset shape: (26, 3)

คอลัมน์ใน Train dataset:
['Subject', 'Body', 'Label']

🔍 การตรวจสอบโครงสร้างข้อมูล
มีคอลัมน์ Subject: True
มีคอลัมน์ Body: True
มีคอลัมน์ Label: True
✅ สร้าง combined_text จาก Subject และ Body

🔧 การ Extract Stylometric Features
กำลัง extract features จาก train dataset...
Processed 0/100 emails...
Processed 10/100 emails...
Processed 20/100 emails...
Processed 30/100 emails...
Processed 40/100 emails...
Processed 50/100 emails...
Processed 60/100 emails...
Processed 70/100 emails...
Processed 80/100 emails...
Processed 90/100 emails...
กำลัง extract features จาก test dataset...
Processed 0/26 emails...
Processed 10/26 emails...
Processed 20/26 emails...

✅ การ extract features เสร็จสมบูรณ์
จำนวน features ที่ได้: 61
จำนวนข้อมูลฝ

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [05:49:40] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [05:49:40] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [05:49:40] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [05:49:40] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [05:49:40] WARNING: /w

...

<IPython.core.display.Javascript object>

  ✅ ดาวน์โหลด xgb_feature_importance.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ✅ ดาวน์โหลด ablation_study_results.csv

🎯 สรุปผลการทดลอง
🏆 โมเดลที่ดีที่สุด: XGBoost
📊 ความแม่นยำสูงสุด: 0.9231
🔢 จำนวน features ที่ใช้: 61
📁 จำนวนข้อมูลฝึก: 100
📁 จำนวนข้อมูลทดสอบ: 26
🏷️  คลาส: ['Legitimate', 'Phishing']

💾 ไฟล์ที่สร้าง:
  ✅ stylometric_features_train.csv - Features จากข้อมูลฝึก
  ✅ stylometric_features_test.csv - Features จากข้อมูลทดสอบ
  ✅ stylometric_features_train_with_labels.csv - Features + Labels จากข้อมูลฝึก
  ✅ stylometric_features_test_with_labels.csv - Features + Labels จากข้อมูลทดสอบ
  ✅ stylometric_feature_statistics_20251129_054940.csv - สถิติ features
  ✅ model_performance_results.csv - ผลลัพธ์โมเดล
  ✅ xgb_feature_importance.csv - Feature importance (XGBoost)
  ✅ ablation_study_results.csv - ผล ablation study
  ✅ complete_stylometric_analysis_report_20251129_054940.xlsx - รายงานแบบครบถ้วน (รวม Subject และ Body)
  ✅ model_performance_summary_20251129_054941.xlsx - รายงานสรุปผลการทดลอง

📋 ในไฟล์รายงานแบบครบถ้วนจะมี:
  - ข้อมูลต้นฉบับ (Subject, Body) + 